In [0]:
%%sh
# 

# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/temp/ --human --summarize

# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v2/dimension/

aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v4/dimension/date=2020-03-21/

In [0]:
%%sh


PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.26.136  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
\c aa
select count(*),DATE(update_time) AS ForDate
from aa_domain_metadata where DATE(update_time) between '2020-03-24' and '2020-03-31' group by DATE(update_time) order by DATE(update_time) desc;

EOF

In [0]:
%%sh


PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.26.136  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
\c aa
select  * from aa_domain_metadata where DATE(update_time)='2020-04-01' limit 10;

EOF

In [0]:

from datetime import timedelta, date, datetime
from random import randint
import psycopg2
from contextlib import closing


dbinfo = {
    'NAME': 'aa',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': '10.2.26.136',
    'PORT': 6432,
    
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))

def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

# sql = """select cnt from plproxy.execute_select($$select count(1) as cnt from aa_domain_metadata"
#         "where date = '2019-03-07'$$)t(
#                 cnt bigint);"""

sql = """select count(1) from aa_domain_metadata;"""
            
rows, _ = select(sql, None)
print len(rows)
print rows

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings

PG_AA_HOSTS = [('10.2.26.136', 6432)]
PG_AA_NAME = 'aa'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql= """select count(distinct name) from aa_domain_metadata;"""
def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
print query(aa_dsn, sql)

# res = pg_settings(urn, dis, None, 'aa')

In [0]:

import json
import random
import time

def time_cost(func):
    def test_wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print "{} cost {} second".format(func.__name__, round(end - start,2))
        return result
    return test_wrapper

@time_cost
def test_missing_date():
    date_list = ['2019-03-19', '2019-03-22', '2019-03-26', '2019-04-27', '2019-05-04', '2019-05-11', '2019-05-18', '2019-05-25', '2019-05-31']
    for _date in date_list:
        d = spark.read.text("s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/temp/{}.deflate/".format(_date)).collect()
        _json = json.loads(d[0].value)
        raw_dic = _json[0]['source'][0]['raw_data']
        random_key = random.sample(raw_dic.keys(), 5)
    # test count
        raw_count = len(raw_dic.keys())
        unified_count = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v4/dimension/date={}/".format(_date)).distinct().count()
        try:
            assert raw_count == unified_count, "Count mismatched in {}".format(_date)
        except AssertionError as count_msg:
            print count_msg
    
        for _k in random_key:
            # print _k,raw_dic[_k]
            for _detail_col, _detail in raw_dic[_k].items():
              raw_detail = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v4/dimension/date={}/".format(_date)).filter("name='{}'".format(_k)).collect()[0][_detail_col]
              try:
                  assert raw_detail == _detail, "Detail {}'s {} mismatched in {}".format(_k, _detail_col, _date)
              except AssertionError as msg:
                  print msg
    print "ALL DONE"



test_missing_date()



In [0]:

df1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v2/dimension/date=2019-05-31/").filter("name='letrasdemusicas.fm'").drop("_identifier").distinct()



df2 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v4/dimension/date=2019-05-31/").filter("name='letrasdemusicas.fm'").drop("_identifier","date").distinct()

df1.show()
df2.show()


# df3 = df1.join(df2,["name", "description", "original_icon_url", "icon_url", "site_url", "path"],how='inner')
# df3.show()
# cnt = df3.count()
# print cnt

In [0]:

from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
from collections import defaultdict
import re
import time
import random
# Test Count
# v1:2018.09.18～2019.03.15
# v2:2019.05.31～2020.03.07

def time_cost(func):
    def test_wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print "{} cost {} second".format(func.__name__, round(end - start,2))
        return result
    return test_wrapper

ORIGIN_PATH = "s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.{key}/dimension/date={date}/"
TARGET_PATH = "s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v4/dimension/date={date}/"
def _parse_string(path):
    pattern = re.compile("unified.*?date=(.*?)/")
    return pattern.search(path).groups()[0]

def get_bucket_date_range():
    date_range = defaultdict(list)
    
    s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-mobileweb-free'))
    for _v in ["v1", "v2"]:
        path_list = s3_bucket_list.all(prefix="unified/mobileweb.domain.{}/dimension/".format(_v), depth_is_1=True)
        for _path in path_list:
           date_range[_v].append(_parse_string(_path))
    return date_range
    
def _prepare_name(key, date):
    name_list = spark.read.parquet(ORIGIN_PATH.format(key=key, date=date)).select("name").collect()
    return random.sample(name_list, 1000)
    
@time_cost
def test_mobile():
    all_date = get_bucket_date_range()
    for _k, _v in all_date.items():
        for date_ in _v:
            start_time = time.time()
            unified_df_cnt = spark.read.parquet(ORIGIN_PATH.format(key=_k, date=date_)).count()
            delta_df_cnt = spark.read.format("delta").load(TARGET_PATH.format(date=date_)).count()
            assert unified_df_cnt == delta_df_cnt, "Count Not Equal in {}".format(date_)
            
            # check detail
            _test_details(_k, date_)
            # print("{} cost {} seconds".format(date_, (time.time() - start_time)))



# @time_cost
def _test_details(key, date):
    V1_COLUMN = ["description", "icon_url", "name", "original_icon_url", "site_url"]
    V2_COLUMN = ["description", "icon_url", "name", "original_icon_url", "path", "site_url" ]
    col_name_list = _prepare_name(key, date)
    for _name in col_name_list:
        df_1 = spark.read.parquet(ORIGIN_PATH.format(key=key, date=date)).filter("name='{}'".format(_name[0])).drop("_identifier").distinct()
        df_2 = spark.read.format("delta").load(TARGET_PATH.format(date=date)).filter("name='{}'".format(_name[0])).drop("date","_identifier").distinct()
        if key == 'v1':
            df_3 = df_1.join(df_2, V1_COLUMN, how = 'inner')
        elif key == 'v2':
            df_3 = df_1.join(df_2, V2_COLUMN, how = 'inner')
        # print "{} is under asserting ...".format(_name[0])
        # df_1.show()
        # df_2.show()
        # df_3.show()
        # print df_3.count()
        try:
            assert df_1.count() == 1, "Name: '{}' in {} raw data is not unified".format(_name[0], date)
            assert df_1.count() == 1, "Name: '{}' in {} delta lake data is not unified".format(_name[0], date)
            assert df_3.count() == 1, "Name: '{}' mismatched in {}".format(_name[0], date)
        except AssertionError as msg:
            print msg



# test_mobile()
# test_mobile()
# start_time = time.time()
# _test_details("v2", "2019-05-31")
# print("{} cost {} seconds".format('2018-09-18', (time.time() - start_time)))
_prepare_name("v1", "2018-09-18")


In [0]:
%%sh


PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.26.136  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
\c aa
\d aa_domain_metadata
select  * from aa_domain_metadata where DATE(update_time)='2020-04-01' limit 10;

EOF

In [0]:

r = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v4/dimension/date=2020-03-21/").filter("name='{NAME}'".format(NAME='couples.jp')).select("description", "original_icon_url", "site_url").collect()
print r[0][0]
DETAIN_SQL = """SELECT description,original_icon_url,site_url FROM aa_domain_metadata WHERE name='couples.jp';"""
res = query(aa_dsn, DETAIN_SQL)
print res[0][0]

In [0]:



DETAIN_SQL = """SELECT description,original_icon_url,site_url FROM aa_domain_metadata WHERE name='{}';"""

for _name in _prepare_name("v4", "2020-03-21"):
    # print _name[0]
    db_res = query(aa_dsn, DETAIN_SQL.format(_name[0]))
    des,icon,sit_u =  (db_res[0][0]), (db_res[0][1]), (db_res[0][2])
    un_res = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v4/dimension/date=2020-03-21/").filter("name='{NAME}'".format(NAME=_name[0])).select("description", "original_icon_url", "site_url").collect()
    try:
        assert icon == un_res[0][1], "{}, {}".format(des, un_res[0][1])
        assert sit_u == un_res[0][2], "{}, {}".format(des, un_res[0][2])
    except AssertionError as m:
        print m
        
        
    

    
